In [1]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
#그래프를 노트북 안에 그리기 위해서 설정

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [30]:
################
# Layer Params #
################
n_steps = 54
n_inputs = 54
n_neurons = 150
n_outputs = 4


train_input = np.load('rnn_train_input.npy')
train_label = np.load('rnn_train_label.npy')

test_input = np.load('rnn_test_input.npy')
test_label = np.load('rnn_test_label.npy')

#true값 중 최대값: 543
#true값 중 최소값: 72
#543-72=471

train_input = (train_input.astype(np.float32).reshape(-1, 54*54) -72) / 471.0  #/ 255.0  # (784,)
test_input = (test_input.astype(np.float32).reshape(-1, 54*54) -72) / 471.0  #/ 255.0

train_label = train_label.astype(np.int32)
test_label = test_label.astype(np.int32)

train_input = train_input.reshape([-1, n_steps, n_inputs])
test_input = test_input.reshape([-1, n_steps, n_inputs])

print('train_input.shape :', train_input.shape)
print('test_input.shape :', test_input.shape)

train_input.shape : (1288, 54, 54)
test_input.shape : (552, 54, 54)


In [38]:
train_input[140]

array([[0.35456476, 0.3524416 , 0.35031846, ..., 0.7346072 , 0.7324841 ,
        0.7282378 ],
       [0.72611463, 0.7239915 , 0.72186834, ..., 0.45010614, 0.44585988,
        0.44161358],
       [0.4373673 , 0.43312103, 0.42675158, ..., 0.3821656 , 0.38641188,
        0.39278132],
       ...,
       [0.36305732, 0.36093417, 0.35881105, ..., 0.7197452 , 0.7133758 ,
        0.7091295 ],
       [0.7027601 , 0.6963906 , 0.6878981 , ..., 0.42887473, 0.42675158,
        0.42462844],
       [0.42250532, 0.42038217, 0.41825902, ..., 0.43312103, 0.4373673 ,
        0.43949044]], dtype=float32)

In [39]:
type(train_label[4])

numpy.int32

In [40]:
def shuffle_batch(features, labels, batch_size):
    rnd_idx = np.random.permutation(len(features))
    n_batches = len(features) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_x, batch_y = features[batch_idx], labels[batch_idx]
        yield batch_x, batch_y

In [45]:
reset_graph()

inputs = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
labels = tf.placeholder(tf.int32, [None])

# RNN Model
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, inputs, dtype=tf.float32)
# dense layer
logits = tf.layers.dense(states, n_outputs)  # states = outputs[-1]

# loss
xentropy = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

################
# Train Params #
################
learning_rate = 0.0001
n_epochs = 50
batch_size = 250

# optimizer
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(xentropy)

# metric
correct = tf.nn.in_top_k(logits, labels, 1)
#evaluation() 함수는 단순히 tf.nn.in_top_kop를 생성한다. 이 op는 자동적으로 참인 레이블이 K most-likely 예측에서 발견되면, 각 모델의 출력을 올바르다고 채점한다.
#이 경우에 참인 레이블에 대해 예측이 옳았을 경우만 K의 값을 1로 설정합니다.

accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [46]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for i in range(0, len(train_input), batch_size):
            start = i
            if start + batch_size > len(train_input) - 1:
                end = len(train_input) - 1
            else:
                end = start + batch_size
            batch_x = train_input[start:end]
            batch_y = train_label[start:end]
            sess.run(train_op, feed_dict={inputs: batch_x, labels: batch_y})
        acc_batch = accuracy.eval(feed_dict={inputs: batch_x, labels: batch_y})
        acc_valid = accuracy.eval(feed_dict={inputs: test_input, labels: test_label})
        loss_batch = xentropy.eval(feed_dict={inputs: batch_x, labels: batch_y})
        print('epoch : {:03d}'.format(epoch),
              'acc_batch : {:.4f}, acc_valid : {:.4f}'.format(acc_batch, acc_valid),
              'loss_batch : {:.4f}'.format(loss_batch))

epoch : 000 acc_batch : 0.1622, acc_valid : 0.0598 loss_batch : 1.4168
epoch : 001 acc_batch : 0.3243, acc_valid : 0.2500 loss_batch : 1.3646
epoch : 002 acc_batch : 0.4054, acc_valid : 0.3007 loss_batch : 1.3530
epoch : 003 acc_batch : 0.2973, acc_valid : 0.2337 loss_batch : 1.3580
epoch : 004 acc_batch : 0.1622, acc_valid : 0.0851 loss_batch : 1.3664
epoch : 005 acc_batch : 0.3514, acc_valid : 0.0942 loss_batch : 1.3755
epoch : 006 acc_batch : 0.3784, acc_valid : 0.0942 loss_batch : 1.3742
epoch : 007 acc_batch : 0.3243, acc_valid : 0.0978 loss_batch : 1.3653
epoch : 008 acc_batch : 0.4324, acc_valid : 0.1123 loss_batch : 1.3613
epoch : 009 acc_batch : 0.5676, acc_valid : 0.1395 loss_batch : 1.3619
epoch : 010 acc_batch : 0.4054, acc_valid : 0.1051 loss_batch : 1.3618
epoch : 011 acc_batch : 0.3243, acc_valid : 0.0833 loss_batch : 1.3613
epoch : 012 acc_batch : 0.3243, acc_valid : 0.0761 loss_batch : 1.3618
epoch : 013 acc_batch : 0.3243, acc_valid : 0.0815 loss_batch : 1.3593
epoch 

In [ ]:
#correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy:', sess.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))